In [15]:
import numpy as np
import pandas as pd
import gc
import sys
from surprise import Dataset
from sklearn.preprocessing import MinMaxScaler
from surprise import SVD
from surprise import Reader
from surprise.model_selection import train_test_split 

In [17]:
df_steam_games = pd.read_csv('../datasets/df_steam_games_clean.csv')
df_user_reviews = pd.read_csv('../datasets/df_user_reviews_clean.csv')
df_user_items = pd.read_csv('../datasets/df_user_items_clean.csv.gz',compression='gzip')

In [22]:
df_user_reviews.to_csv("user_reviews_ur.csv", index=False)

In [24]:
merged_df = df_steam_games.merge(df_user_reviews, left_on='id', right_on='item_id', how='inner')

In [13]:
df_user_reviews = merged_df

In [25]:
merged_df.head()

,genres,title,release_date,price,id,developer,user_id,item_id,recommend,sentiment_analysis,rating
0,"['Action', 'Indie', 'Racing']",Carmageddon Max Pack,1997,9.99,282010.0,Stainless Games Ltd,InstigatorAU,282010,1.0,0.5,4.0
1,['Action'],Half-Life,1998,9.99,70.0,Valve,EizanAratoFujimaki,70,1.0,1.0,5.0
2,['Action'],Half-Life,1998,9.99,70.0,Valve,GamerFag,70,1.0,0.0,3.0
3,['Action'],Half-Life,1998,9.99,70.0,Valve,76561198020928326,70,1.0,1.0,5.0
4,['Action'],Half-Life,1998,9.99,70.0,Valve,Bluegills,70,1.0,1.0,5.0


In [26]:
para_entrenar = merged_df[['user_id','title','rating']]

In [28]:
reader = Reader(rating_scale=(0, 4))

data = Dataset.load_from_df(para_entrenar, reader)

In [31]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

## creo la columna rating

In [20]:


# creo la columna rating

# Paso 1: Convertir los valores booleanos a enteros
df_user_reviews['recommend'] = df_user_reviews['recommend'].replace({True: 1, False: 0})

# Paso 2: Normalizar las columnas de recomendación y análisis de sentimientos
scaler = MinMaxScaler()
scaler.fit(df_user_reviews[['recommend', 'sentiment_analysis']])
df_user_reviews[['recommend', 'sentiment_analysis']] = scaler.transform(df_user_reviews[['recommend', 'sentiment_analysis']])

# Paso 3: Crear la columna de calificación
df_user_reviews['rating'] = df_user_reviews['recommend'] * 0.6 + df_user_reviews['sentiment_analysis'] * 0.4

# Paso 4: Ajustar la columna de calificación
df_user_reviews['rating'] = df_user_reviews['rating'] * 5

# Verificar el resultado
df_user_reviews["rating"].unique()


array([5., 4., 3., 1., 0., 2.])

In [13]:
df_user_reviews

,user_id,item_id,recommend,sentiment_analysis,rating
0,76561197970982479,1250,1.0,1.0,5.0
1,76561197970982479,22200,1.0,1.0,5.0
2,76561197970982479,43110,1.0,1.0,5.0
3,js41637,251610,1.0,1.0,5.0
4,js41637,227300,1.0,1.0,5.0
...,...,...,...,...,...
59300,76561198312638244,70,1.0,1.0,5.0
59301,76561198312638244,362890,1.0,1.0,5.0
59302,LydiaMorley,273110,1.0,1.0,5.0
59303,LydiaMorley,730,1.0,1.0,5.0


In [ ]:
# nuevo_df = df_user_reviews[['user_id','title','rating']]

In [ ]:
# reader = Reader(rating_scale=(0, 4))

# data = Dataset.load_from_df(nuevo_df, reader)

In [29]:

from surprise import SVD
model = SVD()

In [32]:
# Entrenamos el modelo
model.fit(train_data)

In [33]:
# Predecimos
predictions = model.test(test_data)

In [34]:
predictions[1]

Prediction(uid='76561198074783932', iid='Rocket League®', r_ui=5.0, est=4, details={'was_impossible': False})

In [35]:
# Hacemos una predicción al azar para usuario y juego
model.predict("76561198081529182","Killing Floor")

Prediction(uid='76561198081529182', iid='Killing Floor', r_ui=None, est=4, details={'was_impossible': False})

In [43]:
def recomendacion_usuario(user_id):
  # Obtener todos los juegos que el usuario ha calificado
  juegos_usuario = df_user_reviews[df_user_reviews['user_id'] == user_id]['item_id'].values
  
  # Obtener todos los juegos posibles
  todos_los_juegos = df_user_reviews['item_id'].unique()
  
  # Obtener las predicciones para cada juego que el usuario no ha calificado
  predicciones = [model.predict(user_id, juego) for juego in todos_los_juegos if juego not in juegos_usuario]
  
  # Ordenar las predicciones por puntuación de predicción
  predicciones_ordenadas = sorted(predicciones, key=lambda x: x.est, reverse=True)
  
  # Devolver los 5 juegos recomendados
  return [pred.iid for pred in predicciones_ordenadas[:5]]


In [44]:
recommend_function = recomendacion_usuario("76561198081529182")

In [45]:
recommend_function

[22200, 43110, 251610, 227300, 239030]